In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
import pandas
import json
import requests
from time import sleep
from tqdm.notebook import tqdm, trange

In [3]:
#driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))
driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())



====== WebDriver manager ======
Current chromium version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 chromium
There is no [linux64] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_linux64.zip
Driver has been saved in cache [/home/tim/.wdm/drivers/chromedriver/linux64/98.0.4758.102]
/tmp/ipykernel_2566679/689089049.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())


In [4]:
url_games = 'https://www.boardgamers.space/boardgame/gaia-project/games'

In [5]:
driver.get(url=url_games)

In [6]:
finished_button = driver.find_element_by_partial_link_text("Finished")

/tmp/ipykernel_2566679/2660566465.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  finished_button = driver.find_element_by_partial_link_text("Finished")


In [7]:
finished_button.click()

In [14]:
"""
Combination of below cells to update the game names file

Currently set to run through whole set and checked for missed
"""

#link to click for next page
next_button = driver.find_elements_by_class_name("page-link")[-2]
#parent <li> can be disabled if last page
next_item = driver.find_elements_by_class_name("page-item")[-2]
#list of disabled items
disabled_items = driver.find_elements_by_class_name("disabled")

#list of names items
game_names = driver.find_elements_by_class_name("game-name")

games_set = set()

pbar = tqdm(total=32078)

with open("game_names.txt", "r") as names_file:
    lines = names_file.readlines()
    for line in lines:
        games_set.add(line)

names_file.close()
    
with open("game_names.txt", "a") as names_file:
    
    for name in game_names:
        n = name.text
        if n not in games_set:
            names_file.write(n + '\n' )
    
    while next_item not in disabled_items:
        try:
            next_button.click()
            sleep(.1)

            #wait for JS to load -> elements are same, only text and class are updated
            while driver.execute_script("return document.readyState") != "complete":
                sleep(.05)

            next_button = driver.find_elements_by_class_name("page-link")[-2]
            next_item = driver.find_elements_by_class_name("page-item")[-2]
            disabled_items = driver.find_elements_by_class_name("disabled")

            game_names = driver.find_elements_by_class_name("game-name")

            for name in game_names:
                n = name.text
                if n not in games_set:
                    names_file.write(n + '\n' )
                pbar.update(1)
        except:
            break

names_file.close()

/tmp/ipykernel_2566679/1689032646.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_button = driver.find_elements_by_class_name("page-link")[-2]
/tmp/ipykernel_2566679/1689032646.py:10: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_item = driver.find_elements_by_class_name("page-item")[-2]
/tmp/ipykernel_2566679/1689032646.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  disabled_items = driver.find_elements_by_class_name("disabled")
/tmp/ipykernel_2566679/1689032646.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  game_names = driver.find_elements_by_class_name("game-name")


  0%|          | 0/32078 [00:00<?, ?it/s]

/tmp/ipykernel_2566679/1689032646.py:44: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_button = driver.find_elements_by_class_name("page-link")[-2]
/tmp/ipykernel_2566679/1689032646.py:45: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_item = driver.find_elements_by_class_name("page-item")[-2]
/tmp/ipykernel_2566679/1689032646.py:46: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  disabled_items = driver.find_elements_by_class_name("disabled")
/tmp/ipykernel_2566679/1689032646.py:48: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  game_names = driver.find_elements_by_class_name("game-name")


In [122]:
#link to click for next page
next_button = driver.find_elements_by_class_name("page-link")[-2]
#parent <li> can be disabled if last page
next_item = driver.find_elements_by_class_name("page-item")[-2]
#list of disabled items
disabled_items = driver.find_elements_by_class_name("disabled")

#list of names items
game_names = driver.find_elements_by_class_name("game-name")

with open("game_names.txt", "w") as names_file:
    for name in game_names:
        names_file.write(name.text + "\n")
    names_file.close()

/tmp/ipykernel_3727300/346778655.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_button = driver.find_elements_by_class_name("page-link")[-2]
/tmp/ipykernel_3727300/346778655.py:4: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_item = driver.find_elements_by_class_name("page-item")[-2]
/tmp/ipykernel_3727300/346778655.py:6: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  disabled_items = driver.find_elements_by_class_name("disabled")
/tmp/ipykernel_3727300/346778655.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  game_names = driver.find_elements_by_class_name("game-name")


In [123]:
pbar = tqdm(total=3199)

names_file = open("game_names.txt", "a")

while next_item not in disabled_items:
    try:
        next_button.click()
        sleep(.05)

        #wait for JS to load -> elements are same, only text and class are updated
        while driver.execute_script("return document.readyState") != "complete":
            sleep(.05)

        next_button = driver.find_elements_by_class_name("page-link")[-2]
        next_item = driver.find_elements_by_class_name("page-item")[-2]
        disabled_items = driver.find_elements_by_class_name("disabled")

        game_names = driver.find_elements_by_class_name("game-name")

        for name in game_names:
            names_file.write(name.text + "\n")

        pbar.update(1)
    except:
        break

names_file.close()

  0%|          | 0/3199 [00:00<?, ?it/s]

/tmp/ipykernel_3727300/3469520725.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_button = driver.find_elements_by_class_name("page-link")[-2]
/tmp/ipykernel_3727300/3469520725.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  next_item = driver.find_elements_by_class_name("page-item")[-2]
/tmp/ipykernel_3727300/3469520725.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  disabled_items = driver.find_elements_by_class_name("disabled")
/tmp/ipykernel_3727300/3469520725.py:18: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  game_names = driver.find_elements_by_class_name("game-name")


In [124]:
driver.close()